In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

### Reading files

In [23]:
df = pd.read_csv('data.csv')

### Understanding The Data and Cleaning it

In [24]:
df.head()
df_meeting_data = df.iloc[:1]
df_meeting_data

,Meeting ID,Topic,Start Time,End Time,User Email,Duration (Minutes),Participants,Unnamed: 7
0,94260712307,BTEC Fall Training,11/08/2022 08:57:15 AM,11/08/2022 11:11:34 AM,b.Jaber@ltuc.com,135,64,NaN


In [25]:
df_1 , df_participants = df[:1], df[1:]
meeting_duration= int(df_1["Duration (Minutes)"][0])
meeting_duration

135

In [26]:
#df_participants.head()
#df_participants.info()
df_participants.columns=df_participants.iloc[0]
df_participants.drop(df_participants.index[0],inplace=True)
df_participants

/tmp/ipykernel_3030/1770108436.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_participants.drop(df_participants.index[0],inplace=True)


1,Name (Original Name),User Email,Join Time,Leave Time,Duration (Minutes),Guest,Recording Consent,In Waiting Room
2,Boshra Jaber,b.jaber@ltuc.com,11/08/2022 08:57:15 AM,11/08/2022 09:41:03 AM,44,No,NaN,No
3,Mostafa Albelbeisi,20020480@student.ltuc.com,11/08/2022 08:57:34 AM,11/08/2022 11:11:34 AM,134,No,NaN,No
4,Mohannad Ghabashneh,22034356@student.ltuc.com,11/08/2022 08:57:35 AM,11/08/2022 11:11:34 AM,134,No,NaN,No
5,Abdallah Mousa,22033848@student.ltuc.com,11/08/2022 08:58:00 AM,11/08/2022 11:11:34 AM,134,No,NaN,No
6,Ashar Momani,NaN,11/08/2022 08:58:14 AM,11/08/2022 11:11:34 AM,134,Yes,Yes,No
...,...,...,...,...,...,...,...,...
61,Tahani Ali,NaN,11/08/2022 10:42:45 AM,11/08/2022 10:44:21 AM,2,Yes,NaN,No
62,Tahani Ali,NaN,11/08/2022 10:46:22 AM,11/08/2022 11:11:34 AM,26,Yes,NaN,No
63,رزان عويضات,NaN,11/08/2022 09:36:33 AM,11/08/2022 09:37:59 AM,2,Yes,NaN,No
64,Tahani Ali,NaN,11/08/2022 10:44:22 AM,11/08/2022 10:45:39 AM,2,Yes,NaN,No


In [27]:
df_participants.columns

Index(['Name (Original Name)', 'User Email', 'Join Time', 'Leave Time',
       'Duration (Minutes)', 'Guest', 'Recording Consent', 'In Waiting Room'],
      dtype='object', name=1)

### Analyzing Students Attendance

Specifications:
- student is considered present if they attend the meeting on time or less than 10 minutes late

- student is considered late if they're late more than 20 minutes

- student is considered Absent if they were late more than 20 minutes.

In [28]:
df_participants["Student ID"] = df_participants["User Email"].str.extract(pat='(^[0-9]*)')
df_participants = df_participants.dropna(subset=['Student ID'])
df_participants["Attended Time"] = df_participants["Duration (Minutes)"].astype("int")
df_participants = df_participants.groupby("Student ID", as_index=False)["Attended Time"].sum()
df_participants["Attendance Percentage"] = (df_participants["Attended Time"]/meeting_duration)*100
df_participants["Status"] = ['Absent' if x < meeting_duration - 21 else "Late" if meeting_duration - 20 <= x < meeting_duration-10 else "Present" for x in df_participants["Attended Time"]]
df_participants

/tmp/ipykernel_3030/1745824631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_participants["Student ID"] = df_participants["User Email"].str.extract(pat='(^[0-9]*)')
/tmp/ipykernel_3030/1745824631.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_participants["Attended Time"] = df_participants["Duration (Minutes)"].astype("int")


1,Student ID,Attended Time,Attendance Percentage,Status
0,,551,408.148148,Present
1,19012233,132,97.777778,Present
2,20019322,131,97.037037,Present
3,20019523,120,88.888889,Late
4,20020480,134,99.259259,Present
5,21023784,124,91.851852,Late
6,22032930,130,96.296296,Present
7,22033848,134,99.259259,Present
8,22034140,130,96.296296,Present
9,22034185,107,79.259259,Absent


In [29]:
df_participants = df_participants.iloc[1:]

In [32]:
def highlight_cells(val):
    if val == 100:
        code = (0.3,0.5,1)
        color = matplotlib.colors.hsv_to_rgb(code)
        final = matplotlib.colors.to_hex(color, keep_alpha=False)
    elif val >= 50:
        code = (0.3,0.5,0.5)
        color = matplotlib.colors.hsv_to_rgb(code)
        final = matplotlib.colors.to_hex(color, keep_alpha=False)
    elif val < 50:
        code = (0.3,0.5,0.3)
        color = matplotlib.colors.hsv_to_rgb(code)
        final = matplotlib.colors.to_hex(color, keep_alpha=False)
    else:
        final = ''
    return 'background-color: {}'.format(final)


In [36]:
df_participants.style.applymap(highlight_cells, subset=['Attendance Percentage']).to_excel('data.xlsx', engine='openpyxl')


## you can view the data in the excel file (data.xlsx)